# Data preparation

In [1]:
import numpy as np 
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [13]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('telecom_customer_churn.csv')
df.rename(lambda x: x.lower().strip().replace(' ', '_'), axis='columns', inplace=True)
df.columns

Index(['customer_id', 'gender', 'age', 'married', 'number_of_dependents',
       'city', 'zip_code', 'latitude', 'longitude', 'number_of_referrals',
       'tenure_in_months', 'offer', 'phone_service',
       'avg_monthly_long_distance_charges', 'multiple_lines',
       'internet_service', 'internet_type', 'avg_monthly_gb_download',
       'online_security', 'online_backup', 'device_protection_plan',
       'premium_tech_support', 'streaming_tv', 'streaming_movies',
       'streaming_music', 'unlimited_data', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charge', 'total_charges', 'total_refunds',
       'total_extra_data_charges', 'total_long_distance_charges',
       'total_revenue', 'customer_status', 'churn_category', 'churn_reason'],
      dtype='object')

In [14]:
df.describe(include='all')

,customer_id,gender,age,married,number_of_dependents,city,zip_code,latitude,longitude,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_service,internet_type,avg_monthly_gb_download,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,contract,paperless_billing,payment_method,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue,customer_status,churn_category,churn_reason
count,7043,7043,7043.000000,7043,7043.000000,7043,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,3166,7043,6361.000000,6361,7043,5517,5517.000000,5517,5517,5517,5517,5517,5517,5517,5517,7043,7043,7043,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043,1869,1869
unique,7043,2,NaN,2,NaN,1106,NaN,NaN,NaN,NaN,NaN,5,2,NaN,2,2,3,NaN,2,2,2,2,2,2,2,2,3,2,3,NaN,NaN,NaN,NaN,NaN,NaN,3,5,20
top,0002-ORFBO,Male,NaN,No,NaN,Los Angeles,NaN,NaN,NaN,NaN,NaN,Offer B,Yes,NaN,No,Yes,Fiber Optic,NaN,No,No,No,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,NaN,NaN,NaN,NaN,NaN,NaN,Stayed,Competitor,Competitor had better devices
freq,1,3555,NaN,3641,NaN,293,NaN,NaN,NaN,NaN,NaN,824,6361,NaN,3390,5517,3035,NaN,3498,3088,3095,3473,2810,2785,3029,4745,3610,4171,3909,NaN,NaN,NaN,NaN,NaN,NaN,4720,841,313
mean,NaN,NaN,46.509726,NaN,0.468692,NaN,93486.070567,36.197455,-119.756684,1.951867,32.386767,NaN,NaN,25.420517,NaN,NaN,NaN,26.189958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.596131,2280.381264,1.962182,6.860713,749.099262,3034.379056,NaN,NaN,NaN
std,NaN,NaN,16.750352,NaN,0.962802,NaN,1856.767505,2.468929,2.154425,3.001199,24.542061,NaN,NaN,14.200374,NaN,NaN,NaN,19.586585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.204743,2266.220462,7.902614,25.104978,846.660055,2865.204542,NaN,NaN,NaN
min,NaN,NaN,19.000000,NaN,0.000000,NaN,90001.000000,32.555828,-124.301372,0.000000,1.000000,NaN,NaN,1.010000,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10.000000,18.800000,0.000000,0.000000,0.000000,21.360000,NaN,NaN,NaN
25%,NaN,NaN,32.000000,NaN,0.000000,NaN,92101.000000,33.990646,-121.788090,0.000000,9.000000,NaN,NaN,13.050000,NaN,NaN,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.400000,400.150000,0.000000,0.000000,70.545000,605.610000,NaN,NaN,NaN
50%,NaN,NaN,46.000000,NaN,0.000000,NaN,93518.000000,36.205465,-119.595293,0.000000,29.000000,NaN,NaN,25.690000,NaN,NaN,NaN,21.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.050000,1394.550000,0.000000,0.000000,401.440000,2108.640000,NaN,NaN,NaN
75%,NaN,NaN,60.000000,NaN,0.000000,NaN,95329.000000,38.161321,-117.969795,3.000000,55.000000,NaN,NaN,37.680000,NaN,NaN,NaN,30.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.750000,3786.600000,0.000000,0.000000,1191.100000,4801.145000,NaN,NaN,NaN


### Eliminar los datos que no se van a utilizar en el modelo

In [16]:
df=df.drop(columns=['customer_id','churn_category','churn_reason','total_refunds','zip_code','longitude','latitude','city'])

In [17]:
#Seleccionar solamente las filas de clientes que permanecen y que se dan de baja en el periodo
#Se descartan los que acaban de darse de alta
df = df[(df["customer_status"] == 'Churned') | (df["customer_status"] =='Stayed')]

In [18]:
df.shape

(6589, 30)

### Tratamiento de los valores nulos

Una compañía de telecomunicaciones proporciona básicamente dos servicios: teléfono e internet. Pero no es necesario que todos los clientes contraten ambos servicios. Por tanto, muchos de los valores nulos que se observan se refieren a servicios asociados a alguno de los servicios básicos, que si no se ha contratado, aparecen como nulo.

Por tanto, se van a sustituir los valores nulos de las features relacionadas con el servicio de teléfono por "no phone service" para aquellos clientes que solamente tienen contratado el servicio de internet y por "no internet service" en el caso de clientes que solamente tienen contratado el servicio de teléfono. Si son valores numéricos, se asignará un 0.

In [19]:
df.avg_monthly_long_distance_charges=df.avg_monthly_long_distance_charges.fillna(0.0)

In [20]:
df.multiple_lines=df.multiple_lines.fillna('no phone service')

In [21]:
no_internet=['internet_type','online_security','online_backup','device_protection_plan','premium_tech_support','streaming_tv',
             'streaming_movies','streaming_music','unlimited_data']
df[no_internet]=df[no_internet].fillna('no internet service')

In [22]:
df.avg_monthly_gb_download=df.avg_monthly_gb_download.fillna(0)

In [23]:
df.describe(include='all')

,gender,age,married,number_of_dependents,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_service,internet_type,avg_monthly_gb_download,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,contract,paperless_billing,payment_method,monthly_charge,total_charges,total_extra_data_charges,total_long_distance_charges,total_revenue,customer_status
count,6589,6589.000000,6589,6589.000000,6589.000000,6589.000000,2991,6589,6589.000000,6589,6589,6589,6589.000000,6589,6589,6589,6589,6589,6589,6589,6589,6589,6589,6589,6589.000000,6589.000000,6589.000000,6589.000000,6589.000000,6589
unique,2,NaN,2,NaN,NaN,NaN,5,2,NaN,3,2,4,NaN,3,3,3,3,3,3,3,3,3,2,3,NaN,NaN,NaN,NaN,NaN,2
top,Male,NaN,Yes,NaN,NaN,NaN,Offer B,Yes,NaN,No,Yes,Fiber Optic,NaN,No,No,No,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,NaN,NaN,NaN,NaN,NaN,Stayed
freq,3312,NaN,3318,NaN,NaN,NaN,824,5945,NaN,3019,5245,2934,NaN,3272,2870,2855,3248,2658,2683,2809,4521,3202,3974,3728,NaN,NaN,NaN,NaN,NaN,4720
mean,NaN,46.760510,NaN,0.476097,2.020944,34.499772,NaN,NaN,23.004277,NaN,NaN,NaN,20.876461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.030695,2432.042243,7.169525,798.087251,3235.218082,NaN
std,NaN,16.841362,NaN,0.968499,3.024016,23.968734,NaN,NaN,15.466844,NaN,NaN,NaN,20.406247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.100727,2265.500080,25.803163,853.767972,2854.600036,NaN
min,NaN,19.000000,NaN,0.000000,0.000000,1.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10.000000,18.850000,0.000000,0.000000,21.610000,NaN
25%,NaN,33.000000,NaN,0.000000,0.000000,12.000000,NaN,NaN,9.250000,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.800000,544.550000,0.000000,106.680000,835.450000,NaN
50%,NaN,46.000000,NaN,0.000000,0.000000,32.000000,NaN,NaN,22.920000,NaN,NaN,NaN,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.050000,1563.900000,0.000000,472.680000,2376.450000,NaN
75%,NaN,60.000000,NaN,0.000000,3.000000,57.000000,NaN,NaN,36.440000,NaN,NaN,NaN,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.400000,4003.000000,0.000000,1275.120000,5106.640000,NaN


In [24]:
df.isnull().sum()

gender                                  0
age                                     0
married                                 0
number_of_dependents                    0
number_of_referrals                     0
tenure_in_months                        0
offer                                3598
phone_service                           0
avg_monthly_long_distance_charges       0
multiple_lines                          0
internet_service                        0
internet_type                           0
avg_monthly_gb_download                 0
online_security                         0
online_backup                           0
device_protection_plan                  0
premium_tech_support                    0
streaming_tv                            0
streaming_movies                        0
streaming_music                         0
unlimited_data                          0
contract                                0
paperless_billing                       0
payment_method                    

### Codificación de columnas con valores categóricos

In [25]:
df['customer_status'] = [1 if x == 'Stayed' else 0 for x in df['customer_status']]

In [26]:
# convertir las categorías a valores numéricos con LabelEncoder
df['married'] = LabelEncoder().fit_transform(df['married'])
df['married']

0       1
1       0
2       0
3       1
4       1
       ..
7037    0
7038    0
7039    1
7041    1
7042    1
Name: married, Length: 6589, dtype: int32

In [27]:
def encode_data(dataframe):
    if dataframe.dtype == "object":
        dataframe = LabelEncoder().fit_transform(dataframe)
    return dataframe

data = df.apply(lambda x: encode_data(x))
data.head()

,gender,age,married,number_of_dependents,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_service,internet_type,avg_monthly_gb_download,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,contract,paperless_billing,payment_method,monthly_charge,total_charges,total_extra_data_charges,total_long_distance_charges,total_revenue,customer_status
0,0,37,1,0,2,9,5,1,42.39,0,1,0,16.0,0,1,0,1,1,0,0,1,1,1,1,65.6,593.30,0,381.51,974.81,1
1,1,46,0,0,0,9,5,1,10.69,1,1,0,10.0,0,0,0,0,0,1,1,0,0,0,1,-4.0,542.40,10,96.21,610.28,1
2,1,50,0,0,0,4,4,1,33.65,0,1,2,30.0,0,0,1,0,0,0,0,1,0,1,0,73.9,280.85,0,134.60,415.45,0
3,1,78,1,0,1,13,3,1,27.82,0,1,2,4.0,0,1,1,0,1,1,0,1,0,1,0,98.0,1237.85,0,361.66,1599.51,0
4,0,75,1,0,3,3,5,1,7.38,0,1,2,11.0,0,0,0,1,1,0,0,1,0,1,1,83.9,267.40,0,22.14,289.54,0


In [28]:
#Guardar los datos preprocesados en un csv
data.to_csv('telecom_preprocessed.csv',index=False)